## Plot EVM

### 1. Query

In [1]:
import pandas as pd
from fetch import QueryParam, Query

DB_NAME = "firecrest"   # all previously saved data can be pulled by filters (stdf_name and test_text)
EXTRA_PIPELINE = ""

query_params = [
    QueryParam(
        stdf_name="fc_bathtub_20211025110838.stdf.gz",  # stdf should be in the same folder with this notebook 
        stdf_id="bathtub",   # stdf_id is for display only, not saved in data base
    ),
]

df = Query(DB_NAME).get_df(query_params, EXTRA_PIPELINE, [
    {"$project": 
        {"_id": 0, "key": 1, "ptr_id": "$ptr._id", "v": "$ptr.v", "t_num": "$ptr.t_num", 
         "mod": {"$mod": ["$ptr.t_num", 5]}
        }},
    {"$match": {"t_num": {"$gte": 44}}},
    {"$match": {"t_num": {"$lte": 145}}},
    {"$sort": {"ptr_id": 1}}, 
])

key = df[df["mod"] == 4]["key"].reset_index(drop=True)
evm = df[df["mod"] == 4]["v"].reset_index(drop=True)
pout = df[df["mod"] == 0]["v"].reset_index(drop=True)
df = pd.concat({"evm": evm, "pout": pout, "key": key}, axis=1)
df.head()

,evm,pout,key
0,-44.295998,3.395684,bathtub_1
1,-43.016621,4.412994,bathtub_1
2,-41.225189,5.498752,bathtub_1
3,-39.059502,6.501049,bathtub_1
4,-36.504787,7.493090,bathtub_1


### 2. Plot

In [2]:
from lets_plot import *
from lets_plot.mapping import as_discrete
LetsPlot.setup_html()

ggplot(data=df) \
    + geom_point(mapping = aes(x='pout', y='evm', color="key")) \
    + scale_color_discrete() \
    + ggtitle("tx evm bathtub")